# 📘 Word 文档 → 文档代码化写作 Demo
本 Notebook 演示如何将 Word 文档内容结构化，并用代码自动生成文档。

In [ ]:
# 安装依赖（首次运行时需要）
!pip install pyyaml jinja2

## 第一步：准备数据 (products.yaml)
在传统 Word 文档里，你可能直接写：
```
N706B 模块 AT 指令手册
波特率：115200
命令：AT+CSQ → 查询信号强度
```
现在我们把它转成结构化数据：

In [ ]:
import yaml

yaml_content = """
products:
  - name: N706B
    version: V1.0
    baud_rate: 115200
    commands:
      - cmd: AT+CSQ
        desc: 查询信号强度
        example: "+CSQ: 25,0"
  - name: N58-GL
    version: V2.0
    baud_rate: 921600
    commands:
      - cmd: AT+CGATT
        desc: 网络附着
        example: "+CGATT: 1"
"""

with open("products.yaml", "w", encoding="utf-8") as f:
    f.write(yaml_content)

data = yaml.safe_load(yaml_content)
data

## 第二步：准备模板
用 Jinja2 模板定义文档的结构，而不是直接在 Word 里排版。

In [ ]:
from jinja2 import Template

template_str = """
{{ name }} 模块 AT 指令手册 (版本 {{ version }})
========================================

基础信息
--------
* 型号: {{ name }}
* 波特率: {{ baud_rate }}

支持命令
--------
| 指令   | 功能描述       | 示例响应 |
|--------|----------------|----------|
{% for c in commands %}| {{ c.cmd }} | {{ c.desc }} | {{ c.example }} |
{% endfor %}
"""

template = Template(template_str)

## 第三步：生成文档
循环读取 YAML 数据，用模板渲染输出 Markdown 文本。

In [ ]:
for product in data["products"]:
    output = template.render(**product)
    filename = f"{product['name']}_manual.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)
    print(f"生成文档: {filename}\n")
    print(output)
    print("-"*40)

## 第四步：下一步拓展
- 现在生成的文档是 Markdown (`.md`)，可以用 Pandoc 转 PDF：
  ```bash
  pandoc N706B_manual.md -o N706B_manual.pdf
  ```
- 也可以用 Sphinx/DITA-OT 来生成 HTML 和 PDF。
- 这就是 **结构化编程 + 文档代码化** 的最小实践。